In [28]:
from PandaStation import (
    PandaStation, FindResource, AddPackagePaths, AddRgbdSensors, draw_points, draw_open3d_point_cloud, 
    create_open3d_point_cloud)



# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import open3d as o3d
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    MakeRenderEngineVtk, DepthRenderCamera, RenderCameraCore, CameraInfo, ClippingRange,  DepthImageToPointCloud,
    BaseField, RenderEngineVtkParams, ConnectMeshcatVisualizer, DepthRange, RgbdSensor, MeshcatPointCloudVisualizer,
    LoadModelDirectives, ProcessModelDirectives
    )
from PandaInverseKinematics import PandaInverseKinematics, PandaIKTraj, Waypoint, Trajectory
from RRT import PandaRRTPlanner, PandaRRTompl
from collections import OrderedDict

import matplotlib.pyplot as plt

In [37]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)
parser = Parser(plant)



panda_hand = parser.AddModelFromFile(FindResource("models/welded_panda_hand.urdf"))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName('panda_hand', panda_hand), RigidTransform(RotationMatrix(), [0,0,0]))


plant.Finalize()



meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, prefix="environment", role = Role.kProximity)


diagram = builder.Build()
context = diagram.CreateDefaultContext()



meshcat.load()
diagram.Publish(context)

scene_graph_context = scene_graph.GetMyContextFromRoot(context)

query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
collision_pairs = query_object.ComputePointPairPenetration()
print(collision_pairs)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6027...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7027/static/
Connected to meshcat-server.
[]
